# Create a catalogue of tweets in mongodb

In [1]:
from datetime import datetime
import json
from hdfs import InsecureClient
from pymongo import MongoClient
from pymongo import UpdateOne, InsertOne


# MONGODB
mongo_url = 'mongodb://hadoop-vm.internal.cloudapp.net:27017/ca2.tweets_202004'
database_name = 'ca2'
collection_name = 'tweets_202004'

# HADOOP
hdfs_url = 'http://hadoop-vm.internal.cloudapp.net:9870'
hdfs_directory = "/twitter"

## Connect and create database and tweets collection
Include index on field id for performance

# Read tweets from hadoop

In [ ]:
# Connect to Hadoop
client = InsecureClient(hdfs_url, user='hduser')

# Connect to MongoDB
mongo_client = MongoClient(mongo_url)
db = mongo_client[database_name]
collection = db[collection_name]

# List JSON files in Hadoop directory
json_files = client.list(hdfs_directory)

# Read JSON files and insert into MongoDB
for file_name in json_files:
    if '2023052' in file_name:
        file_path = f"{hdfs_directory}/{file_name}"
        print(f"processing {file_path}")
        with client.read(file_path) as file:
            bulk_operations = []
            for line in file:
                try:
                    tweet = json.loads(line)
                    if tweet['lang'] == 'en':
                        
                        # Convert timestamp_ms (milliseconds since epoch) to ISODate format
                        timestamp_ms = int(tweet.get('timestamp_ms', 0))
                        timestamp = datetime.utcfromtimestamp(timestamp_ms / 1000)

                        tweet_p = {
                            '_id': tweet['id'],
                            'text': tweet.get('text'),
                            'timestamp_ms': timestamp_ms,
                            'timestamp': timestamp,
                            'coordinates': tweet.get('coordinates'),
                        }
                        if 'extended_tweet' in tweet:
                            tweet_p['text'] = tweet['extended_tweet'].get('full_text')

                        # Add tweet using upsert in case duplicates
                        bulk_operations.append(UpdateOne({'_id': tweet['id']}, {'$set': tweet_p}, upsert=True))

                        # Execute bulk 
                        if len(bulk_operations) >= 10000:
                            print("Writting....")
                            collection.bulk_write(bulk_operations)
                            bulk_operations = []
                except:
                    continue

            # Execute remaining bulk operations
            if bulk_operations:
                collection.bulk_write(bulk_operations)


processing /twitter/covid-tweets-20230522233549-1.json
Writting....
Writting....
processing /twitter/covid-tweets-20230522233549-2.json
Writting....
Writting....
processing /twitter/covid-tweets-20230522233549-3.json
Writting....
Writting....
Writting....
processing /twitter/covid-tweets-20230522233549-4.json
Writting....
Writting....
processing /twitter/covid-tweets-20230522233549-5.json
Writting....
Writting....
processing /twitter/covid-tweets-20230522233549-6.json
Writting....
Writting....
processing /twitter/covid-tweets-20230522233549-7.json
Writting....
Writting....
processing /twitter/covid-tweets-20230523064538-1.json
Writting....
processing /twitter/covid-tweets-20230523064538-2.json
Writting....
processing /twitter/covid-tweets-20230523064538-3.json
Writting....
Writting....
processing /twitter/covid-tweets-20230523064538-4.json
Writting....
Writting....
processing /twitter/covid-tweets-20230523064538-5.json
Writting....
Writting....
processing /twitter/covid-tweets-20230523